In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

import torch
import torch.nn as nn

import random
from deap import base, creator, tools, algorithms

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

wDr = os.getcwd()

In [ ]:
wDr

## Define Model

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, output_size)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Check if GPU is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initilaise the model
forward_model = SimpleMLP(5, 2).to(device)

# Load the saved model weights
forward_model.load_state_dict(torch.load(wDr + r"\forward_model_weights_5000_iteration0.pth"))

# Evaluate each model
forward_model.eval()

## Performance evaluation

In [ ]:
concatenated_df = pd.read_csv(wDr + r"\Final_dataset.csv")
#concatenated_df.set_index(['index'], inplace = True)
concatenated_df.drop('Unnamed: 0', axis = 1, inplace = True)
#concatenated_df = concatenated_df[-35:]
concatenated_df

In [ ]:
X = concatenated_df.iloc[:, :5].values
Y = concatenated_df.iloc[:, 5:].values

In [ ]:
X

In [ ]:
df = pd.DataFrame()
df['PS'] = Y[:, 0]
df['SED'] = Y[:, 1]
df['theta_min'] = X[:, 0]
df['theta_max'] = X[:, 1]
df['t_min'] = X[:, 2]
df['t_max'] = X[:, 3]
df['velocity'] = X[:, 4]
df = df.round(6)
#df['velocity'].unique()

df['sigma_minus_U'] = df['PS'] - 10*df['SED']
df.sort_values('sigma_minus_U', inplace=True)
df

In [ ]:
plt.scatter(df['SED'], df['sigma_minus_U'])

In [ ]:
plt.scatter(df['PS'], df['sigma_minus_U'])

In [ ]:
df_prime = df.groupby('t_min').sum().round(5)
df_prime = df_prime.sort_values(['sigma_minus_U'])
df_prime.reset_index(inplace= True)
df_prime = df_prime[['PS', 'SED', 'theta_min', 'theta_max', 't_min', 't_max', 'velocity', 'sigma_minus_U']]
df_prime = df_prime.iloc[:50]
df_prime

In [ ]:
# Given ordered column values
ordered_values = df_prime['t_min']

# Sort DataFrame based on the order of values in column 'B'
df_sorted = df.set_index('t_min').loc[ordered_values].reset_index()

df_sorted = df_sorted[df_sorted['velocity'] == 1]

# Define the desired column order
column_order = ['PS', 'SED', 'theta_min', 'theta_max', 't_min', 't_max', 'velocity', 'sigma_minus_U']

# Reorder the columns
df = df_sorted[column_order]
df

## GA

#### Fix relative density

In [ ]:
# Independant parameters
Platelength_X = 150.0
Platelength_Y = 100.0
type1 = 2
height = 2

# Dependant parameters
L = Platelength_X/type1/5
B = Platelength_Y/type1/2

LHS = 1153

In [ ]:
# Define your analytical expression
def analytical_expression(theta_min, theta_max, t_min, t_max):
    
    #t = torch.linspace(t_min, t_max, 2*type1 + 1)
    #theta = torch.linspace(theta_min, theta_max, 2*type1)

    steps = 2*type1 + 1
    index = torch.arange(steps, dtype=torch.float32).to(device)
    step_size = (t_max - t_min) / (steps - 1)
    t = t_min + step_size * index

    steps = 2*type1
    index = torch.arange(steps, dtype=torch.float32).to(device)
    step_size = (theta_max - theta_min) / (steps - 1)
    theta = theta_min + step_size * index

    RHS = 0
    for i in range(len(t) - 1):  
        #t_inside = torch.linspace(t[i], t[i+1], 4)

        steps = 4
        index = torch.arange(steps, dtype=torch.float32).to(device)
        step_size = (t[i+1] - t[i]) / (steps - 1)
        t_inside = t[i] + step_size * index

        tan_alpha = (2 - torch.abs(torch.cos(theta[i])))/torch.sin(theta[i])
        
        RHS += torch.abs(torch.cos(theta[i]))/(4 * torch.sin(theta[i])) * ((L - t_inside[1])**2 + (L - t_inside[2])**2 + (L - t[i])**2 + (L - t[i+1])**2) + height * ((L - t[i]) + (L - t[i+1])) + 0.5 * ((L - t_inside[1]) + (L - t_inside[2])) * (B - 2*height - torch.abs(torch.cos(theta[i]))/(2*torch.sin(theta[i])) * ((L - t_inside[1]) + (L - t_inside[2])) - tan_alpha * (t_inside[1]/2 + t_inside[2]/2))

    return RHS

# Define the objective function
def rho_objective(params, target_value):

    theta_min = params[0]
    theta_max = params[1]
    t_min = params[2]
    t_max = params[3]

    return abs(analytical_expression(theta_min, theta_max, t_min, t_max) - target_value)

#### Run

In [ ]:
PS_pred, SED_pred = 0, 0
good = []

while len(good) < 5:
    # Define upper and lower bounds for lattice parameters
    lower_bounds = [np.pi/4, np.pi/4, 0.5, 0.5]  # Lower bounds for each parameter
    upper_bounds = [np.pi/2, np.pi/2, 3, 3]  # Upper bounds for each parameter



    # Step 1: Create the fitness function and individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimization problem (minimizing the difference)
    creator.create("Individual", list, fitness=creator.FitnessMin)




    # Step 2: Initialize population and individual with 4 parameters from DataFrame, respecting bounds
    def create_individual(index):
        #individual = list(df.iloc[index, 2:6].round(6))  # Select lattice parameters from columns 3 to 6
        individual = list(df.iloc[index, 2:6].round(6))  # Select lattice parameters from columns 3 to 6
        return individual

    toolbox = base.Toolbox()
    #toolbox.register("individual", tools.initIterate, creator.Individual, lambda: create_individual(index))
    toolbox.register("individual", tools.initIterate, creator.Individual, lambda: create_individual(random.randint(0, len(df)-1)))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)




    # Step 3: Define evaluation function (difference between the first two columns)
    def evaluate(individual):
        # Find the index in the DataFrame that corresponds to the individual (assuming exact match)
        v1 = torch.tensor([0], dtype=torch.float32).to(device)
        v2 = torch.tensor([0.137931], dtype=torch.float32).to(device)
        v3 = torch.tensor([0.310345], dtype=torch.float32).to(device)
        v4 = torch.tensor([0.482759], dtype=torch.float32).to(device)
        v5 = torch.tensor([0.655172], dtype=torch.float32).to(device)
        v6 = torch.tensor([0.827586], dtype=torch.float32).to(device)
        v7 = torch.tensor([1], dtype=torch.float32).to(device)

        lattice_params = torch.tensor(individual, dtype=torch.float32).to(device)
        #lattice_params = torch.clamp(lattice_params, torch.tensor(lower_bounds, dtype = torch.float32).to(device), torch.tensor(upper_bounds, dtype = torch.float32).to(device))
        #print(lattice_params)

        tensor1 = torch.concat((lattice_params, v1))
        tensor2 = torch.concat((lattice_params, v2))
        tensor3 = torch.concat((lattice_params, v3))
        tensor4 = torch.concat((lattice_params, v4))
        tensor5 = torch.concat((lattice_params, v5))
        tensor6 = torch.concat((lattice_params, v6))
        tensor7 = torch.concat((lattice_params, v7))
        
        outputs1 = forward_model(tensor1).detach().cpu().numpy()
        outputs2 = forward_model(tensor2).detach().cpu().numpy()
        outputs3 = forward_model(tensor3).detach().cpu().numpy()
        outputs4 = forward_model(tensor4).detach().cpu().numpy()
        outputs5 = forward_model(tensor5).detach().cpu().numpy()
        outputs6 = forward_model(tensor6).detach().cpu().numpy()
        outputs7 = forward_model(tensor7).detach().cpu().numpy()

        return (outputs1[0] - 10*outputs1[1] + outputs2[0] - 10*outputs2[1] + outputs3[0] - 10*outputs3[1] + outputs4[0] - 10*outputs4[1] +
                    outputs5[0] - 10*outputs5[1] + outputs6[0] - 10*outputs6[1] + outputs7[0] - 10*outputs7[1] + rho_objective(lattice_params, LHS),)




    # Step 4: Define bounded mutation to keep individuals within bounds
    def bounded_mutate(individual, mu, sigma, indpb):
        for i in range(len(individual)):
            if random.random() < indpb:
                individual[i] += random.gauss(mu, sigma)
                # Apply bounds to each parameter
                individual[i] = min(max(individual[i], lower_bounds[i]), upper_bounds[i])
        return individual,

    def apply_bounds(individual, lower_bounds, upper_bounds):
        return [min(max(ind, lb), ub) for ind, lb, ub in zip(individual, lower_bounds, upper_bounds)]

    def crossover_and_clamp(ind1, ind2, alpha=0.5):
        tools.cxBlend(ind1, ind2, alpha)
        ind1[:] = apply_bounds(ind1, lower_bounds, upper_bounds)
        ind2[:] = apply_bounds(ind2, lower_bounds, upper_bounds)
        return ind1, ind2




    # Step 5: Register genetic operators
    #toolbox.register("mate", tools.cxBlend, alpha=0.5)  # Blend crossover
    toolbox.register("mate", crossover_and_clamp)
    toolbox.register("mutate", bounded_mutate, mu=0, sigma=0.2, indpb=0.2)  # Bounded mutation
    toolbox.register("select", tools.selTournament, tournsize=3)  # Tournament selection
    toolbox.register("evaluate", evaluate)




    # Step 6: Define genetic algorithm parameters
    population_size = 50  # Size of the population
    num_generations = 50  # Number of generations
    cx_prob = 0.9  # Crossover probability
    mut_prob = 0 #0.2  # Mutation probability




    # Step 7: Run the Genetic Algorithm

    # Initialize population
    #population = toolbox.population(n=population_size)
    population = [creator.Individual(create_individual(i)) for i in range(len(df))]

    # Statistics to track progress
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda pop: sum(fitness[0] for fitness in pop) / len(pop))
    stats.register("min", lambda pop: min(fitness[0] for fitness in pop))
    stats.register("max", lambda pop: max(fitness[0] for fitness in pop))

    # Run the genetic algorithm
    #population, logbook = algorithms.eaSimple(population, toolbox, cxpb=cx_prob, mutpb=mut_prob,
        #                                           ngen=num_generations, stats=stats, verbose=True)


    # Custom loop to store best individual each generation
    best_individuals_per_gen = []

    # Initialize lists to store fitness values for each generation
    min_fitness_values = []
    max_fitness_values = []
    avg_fitness_values = []

    for gen in range(num_generations):

        # Perform one generation
        offspring = algorithms.varAnd(population, toolbox, cxpb=cx_prob, mutpb=mut_prob)
        fits = list(map(toolbox.evaluate, offspring))
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit

        # Select the next generation population
        population = toolbox.select(offspring, k=len(population))

        # Calculate fitness statistics
        min_fitness = min(ind.fitness.values[0].cpu() for ind in population)
        max_fitness = max(ind.fitness.values[0].cpu() for ind in population)
        avg_fitness = sum(ind.fitness.values[0].cpu() for ind in population) / len(population)

        # Store fitness statistics for plotting later
        min_fitness_values.append(min_fitness)
        max_fitness_values.append(max_fitness)
        avg_fitness_values.append(avg_fitness)

        # Get the best individual of the current generation
        best_individual = tools.selBest(population, k=1)[0]
        best_individuals_per_gen.append(best_individual)

        # Print best individual and fitness
        #print(f"Generation {gen}: Best Individual = {best_individual}, Fitness = {best_individual.fitness.values[0]}")

    # Save the best individuals from each generation
    best_individuals_df = pd.DataFrame([list(ind) for ind in best_individuals_per_gen])
    best_individuals_df.to_csv("best_individuals_per_gen" + str(gen) + ".csv", index=False)


    # Output the best individual found
    best_individual = tools.selBest(population, k=10)

    optimized_params = pd.DataFrame(np.array(best_individual))

    lattice_params = torch.tensor(best_individual, dtype=torch.float32).to(device)

    good.append(lattice_params)

# Plot the fitness values for each generation
fig, ax = plt.subplots()
#plt.figure(figsize=(10, 6))

# Adjust tick font sizes
ax.tick_params(axis='both', which='major', labelsize=12)  # Change fontsize for major ticks
ax.tick_params(axis='both', which='minor', labelsize=10)  # Optional: Change fontsize for minor ticks

# Remove the top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


plt.plot(range(num_generations), min_fitness_values, label="Min Fitness", color='red')
plt.plot(range(num_generations), max_fitness_values, label="Max Fitness", color='green')
plt.plot(range(num_generations), avg_fitness_values, label="Avg Fitness", color='blue')
plt.xlabel("Generation", fontsize = 14, fontweight='bold')
plt.ylabel("Fitness", fontsize = 14, fontweight='bold')
#plt.title("Fitness Evolution Over Generations")
plt.legend()
#plt.grid(True)
plt.show()

In [ ]:
PS_pred1, SED_pred1, PS_pred2, SED_pred2, PS_pred3, SED_pred3, PS_pred4, SED_pred4, PS_pred5, SED_pred5, PS_pred6, SED_pred6, PS_pred7, SED_pred7 = [], [], [], [], [], [], [], [], [], [], [], [], [], []


for lattice in good:
    for lat in lattice:
        forward_model.eval()

        v1 = torch.tensor([0], dtype=torch.float32).to(device)
        v2 = torch.tensor([0.137931], dtype=torch.float32).to(device)
        v3 = torch.tensor([0.310345], dtype=torch.float32).to(device)
        v4 = torch.tensor([0.482759], dtype=torch.float32).to(device)
        v5 = torch.tensor([0.655172], dtype=torch.float32).to(device)
        v6 = torch.tensor([0.827586], dtype=torch.float32).to(device)
        v7 = torch.tensor([1], dtype=torch.float32).to(device)

        lattice_params = torch.tensor(lat, dtype=torch.float32).to(device)

        tensor1 = torch.concat((lattice_params, v1))
        tensor2 = torch.concat((lattice_params, v2))
        tensor3 = torch.concat((lattice_params, v3))
        tensor4 = torch.concat((lattice_params, v4))
        tensor5 = torch.concat((lattice_params, v5))
        tensor6 = torch.concat((lattice_params, v6))
        tensor7 = torch.concat((lattice_params, v7))
        
        PS1, SED1 = forward_model(tensor1)
        PS2, SED2 = forward_model(tensor2)
        PS3, SED3 = forward_model(tensor3)
        PS4, SED4 = forward_model(tensor4)
        PS5, SED5 = forward_model(tensor5)
        PS6, SED6 = forward_model(tensor6)
        PS7, SED7 = forward_model(tensor7)

        PS_pred1.append(PS1.detach().cpu().numpy())
        SED_pred1.append(SED1.detach().cpu().numpy())

        PS_pred2.append(PS2.detach().cpu().numpy())
        SED_pred2.append(SED2.detach().cpu().numpy())

        PS_pred3.append(PS3.detach().cpu().numpy())
        SED_pred3.append(SED3.detach().cpu().numpy())

        PS_pred4.append(PS4.detach().cpu().numpy())
        SED_pred4.append(SED4.detach().cpu().numpy())

        PS_pred5.append(PS5.detach().cpu().numpy())
        SED_pred5.append(SED5.detach().cpu().numpy())

        PS_pred6.append(PS6.detach().cpu().numpy())
        SED_pred6.append(SED6.detach().cpu().numpy())

        PS_pred7.append(PS7.detach().cpu().numpy())
        SED_pred7.append(SED7.detach().cpu().numpy())
        #print(PS, SED)

In [ ]:
plt.scatter(PS_pred1, SED_pred1, s=10, color = 'brown')
plt.scatter(PS_pred2, SED_pred2, s=10, color = 'cyan')
plt.scatter(PS_pred3, SED_pred3, s=10, color = 'blue')
plt.scatter(PS_pred4, SED_pred4, s=10, color = 'red')
plt.scatter(PS_pred5, SED_pred5, s=10, color = 'indigo')
plt.scatter(PS_pred6, SED_pred6, s=10, color = 'green')
plt.scatter(PS_pred7, SED_pred7, s=10, color = 'black')

plt.xlim(0, 6)
plt.ylim(0, 0.8)
#plt.legend(['v = 15', 'v = 20', 'v = 25'])
#plt.savefig('optimized1_dynamic.png', transparent = True)
plt.show()

In [ ]:
for i in range(len(good)):
    good[i] = good[i].cpu().numpy()

In [ ]:
reshaped_list = [arr[0].reshape(4) for arr in good]
reshaped_list

In [ ]:
optimized_params = optimized_params.drop_duplicates(subset=[0, 1, 2, 3])
optimized_params

In [ ]:
#optimized_params.to_csv('gradient_optimized_structures_multi_speed_Dynamic_iteration0.csv', header = None, index = None)